<a href="https://colab.research.google.com/github/ngoda/Conversations/blob/master/Chapter_7_Convolution_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copying Files from Google Drive to Colab Parent Directory

In [ ]:
#Copy Files & Directories from one source to destinition

import shutil
import os

# Define the source directory (child folder) and the destination directory (parent path)
source_dir = '/content/drive/MyDrive/Colab Notebooks/data2'
destination_dir = '/content'

# List the contents of the source directory
source_contents = os.listdir(source_dir)

# Copy each file/folder from the source directory to the destination directory
for item in source_contents:
    # Form the source and destination paths
    source_path = os.path.join(source_dir, item)
    destination_path = os.path.join(destination_dir, item)

    # Check if the item is a file or a directory
    if os.path.isfile(source_path):
        # Use shutil.copy() to copy files
        shutil.copy(source_path, destination_path)
    elif os.path.isdir(source_path):
        # Use shutil.copytree() to copy directories recursively
        shutil.copytree(source_path, destination_path)

print("Folder copied successfully from child folder to parent path.")

Folder copied successfully from child folder to parent path.


**Convolution Neural Networks**

 **Implementing a convolution Layer**

 The code generates random image tensors, converts them to column representations with the im2col function, then outputs the resulting shapes.

In [ ]:
# Importing necessary libraries and modules
import numpy as np  # Importing numpy and abbreviating it as np
import sys, os  # Importing sys and os modules

# Appending the parent directory to the system path
sys.path.append(os.pardir)

# Importing the im2col function from the common.util module
from common.util import im2col

# Generating a random input image tensor x1 with shape (1, 3, 7, 7)
x1 = np.random.rand(1, 3, 7, 7)

# Converting the input image tensor x1 into a column representation using im2col
col1 = im2col(x1, 5, 5, stride=1, pad=0)

# Printing the shape of the column representation col1
print(col1.shape)  # Expected output: (9,75)

# Generating a random input image tensor x2 with shape (10, 3, 7, 7)
x2 = np.random.rand(10, 3, 7, 7)

# Converting the input image tensor x2 into a column representation using im2col
col2 = im2col(x2, 5, 5, stride=1, pad=0)

# Printing the shape of the column representation col2
print(col2.shape)  # Expected output: (90,75)


(9, 75)
(90, 75)


**Implementing a convolution Layer**

Code2 - 9 collectively represent the construction and functionality of a simple convolutional neural network for image classification.

In [ ]:

# Code #2
#Defines the Convolution class responsible for performing convolution operations.
#It includes methods for initialization (__init__) and forward pass computation (forward).
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        # Initialize the Convolutional layer with weights, biases, stride, and padding
        self.W = W  # Weights
        self.b = b  # Biases
        self.stride = stride  # Stride
        self.pad = pad  # Padding

    def forward(self, x):
        # Perform forward pass computation of the Convolutional layer
        FN, C, FH, FW = self.W.shape  # Number of filters, Number of channels, Filter height, Filter width
        N, C, H, W = x.shape  # Batch size, Number of channels, Input height, Input width
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)  # Compute output height
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)  # Compute output width
        col = im2col(x, FH, FW, self.stride, self.pad)  # Convert input to column matrix
        col_W = self.W.reshape(FN, -1).T  # Reshape filters to columns
        out = np.dot(col, col_W) + self.b  # Perform convolution operation
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)  # Reshape output and transpose dimensions
        return out




# Code #3
#Defines the col2im function, which performs the reverse operation of im2col,
#converting column representations back to the original input shape.
def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    # Unpack input_shape into variables
    N, C, H, W = input_shape
    # Compute output height and width
    out_h = (H + 2*pad - filter_h) // stride + 1
    out_w = (W + 2*pad - filter_w) // stride + 1
    # Reshape col into appropriate shape and transpose dimensions
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    # Initialize output image array with zeros
    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    # Iterate over filter height
    for y in range(filter_h):
        # Calculate the maximum y position for the current stride
        y_max = y + stride*out_h
        # Iterate over filter width
        for x in range(filter_w):
            # Calculate the maximum x position for the current stride
            x_max = x + stride*out_w
            # Accumulate values from col into img using slicing
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]
    # Return the cropped image without padding
    return img[:, :, pad:H+pad, pad:W+pad]



# Code #4
#Defines the Pooling class for implementing pooling operations (specifically max pooling).
#It includes methods for forward pass computation (forward) and backward pass computation (backward).
class Pooling:
    def __init__(self, pool_h, pool_w, stride=2, pad=0):
        # Initialize the Pooling layer with pool height, pool width, stride, and padding
        self.pool_h = pool_h  # Pool height
        self.pool_w = pool_w  # Pool width
        self.stride = stride  # Stride
        self.pad = pad  # Padding
        self.x = None  # Initialize input variable
        self.arg_max = None  # Initialize variable to store indices of max values

    def forward(self, x):
        # Perform forward pass computation of the Pooling layer
        N, C, H, W = x.shape  # Batch size, Number of channels, Input height, Input width
        out_h = int(1 + (H - self.pool_h) / self.stride)  # Compute output height
        out_w = int(1 + (W - self.pool_w) / self.stride)  # Compute output width
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)  # Convert input to column matrix
        col = col.reshape(-1, self.pool_h*self.pool_w)  # Reshape columns
        arg_max = np.argmax(col, axis=1)  # Find indices of max values
        out = np.max(col, axis=1)  # Perform max pooling operation
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)  # Reshape output and transpose dimensions
        self.x = x  # Save input for backward pass
        self.arg_max = arg_max  # Save indices of max values for backward pass
        return out

    def backward(self, dout):
        # Perform backward pass computation of the Pooling layer
        dout = dout.transpose(3, 2, 1, 0)  # Transpose input gradient
        pool_size = self.pool_h * self.pool_w  # Compute pool size
        dmax = np.zeros((dout.size, pool_size))  # Initialize array for gradients
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()  # Set gradients at max indices
        dmax = dmax.reshape(dout.shape + (pool_size,))  # Reshape gradients
        dx = col2im(dmax, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)  # Convert to image shape
        return dx  # Return gradient of the input



# Code #5
#Defines the SimpleConvNet class, which represents a simple convolutional neural network.
#It includes initialization and methods for prediction (predict), loss computation (loss), and gradient computation (gradient).
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28), conv_param={'filter_num':30, 'filter_size':5, 'pad':8, 'stride':1}, hidden_size=100, output_size=10, weight_init_std=0.01):
        # Initialize SimpleConvNet with input dimensions, convolutional parameters, hidden size, output size, and weight initialization standard deviation
        filter_num = conv_param['filter_num']  # Number of filters
        filter_size = conv_param['filter_size']  # Size of filters
        filter_pad = conv_param['pad']  # Padding size
        filter_stride = conv_param['stride']  # Stride size
        input_size = input_dim[1]  # Input size
        conv_output_size = (input_size + 2*filter_pad - filter_size) / filter_stride + 1  # Compute convolution output size
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))  # Compute pooling output size



# Code #6
# Initializes the parameters of the convolutional neural network (CNN) including weights and biases.
        self.params = {}  # Initialize dictionary to store parameters

        # Initialize and randomly initialize the weights and biases for the first convolutional layer
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)  # Filter weights
        self.params['b1'] = np.zeros(filter_num)  # Filter biases

        # Initialize and randomly initialize the weights and biases for the second fully connected layer
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)  # Fully connected layer weights
        self.params['b2'] = np.zeros(hidden_size)  # Fully connected layer biases

        # Initialize and randomly initialize the weights and biases for the third fully connected layer
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)  # Fully connected layer weights
        self.params['b3'] = np.zeros(output_size)  # Fully connected layer biases



# Code #7
#Constructs the layers of the CNN using the defined classes (Convolution, Relu, Pooling, Affine, SoftmaxWithLoss).
#It sets up the layers in order and initializes the last layer as the softmax with loss layer.
        self.layers = OrderedDict()  # Initialize OrderedDict to store layers

        # Add layers to the network architecture
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])  # First Convolutional layer
        self.layers['Relu1'] = Relu()  # First ReLU activation layer
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)  # First Pooling layer
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])  # First Fully Connected (Affine) layer
        self.layers['Relu2'] = Relu()  # Second ReLU activation layer
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])  # Second Fully Connected (Affine) layer
        self.last_layer = SoftmaxWithLoss()  # Softmax with Loss layer


# Code #8
#Defines the predict and loss methods for the CNN, which compute predictions and loss values respectively.
    def predict(self, x):
        # Perform forward pass prediction through all layers in the network
        for layer in self.layers.values():
            x = layer.forward(x)  # Forward pass through each layer
        return x  # Return the predicted output

    def loss(self, x, t):
        # Compute the loss using forward pass prediction
        y = self.predict(x)  # Predict output using the input data
        return self.last_layer.forward(y, t)  # Compute the loss using the predicted output and target



# Code #9
#Defines the gradient method, which computes gradients during backpropagation.
#It computes the loss, performs backward propagation, saves gradients, and returns them.
    def gradient(self, x, t):
        # Compute gradients for network parameters

        # Forward pass to compute loss
        self.loss(x, t)

        # Backward pass to compute gradients
        dout = 1  # Initialize gradient of loss with respect to output
        dout = self.last_layer.backward(dout)  # Backward pass through the last layer

        layers = list(self.layers.values())  # Get all layers in the network
        layers.reverse()  # Reverse the order of layers for backward pass
        for layer in layers:
            dout = layer.backward(dout)  # Backward pass through each layer

        # Save computed gradients
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db  # Gradients for Convolutional layer
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db  # Gradients for first Fully Connected layer
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db  # Gradients for second Fully Connected layer

        return grads  # Return the computed gradients

